My plan is to build the initial df.

Write a for-loop on a counter of 9, to have a total of 1000 posts.

Pull the 'created_utc' on the current oldest post, last on the current df, and make the 'before' param : that value.

Make next 100 posts a new df.
Append them to the original df.

Ultimately, I want to put this into a function so I can switch to different subreddits and pull submissions and comments.

In [59]:
import pandas as pd
import requests

In [60]:
url = 'https://api.pushshift.io'

In [61]:
subm_end = '/reddit/search/submission'
comment_end = '/reddit/search/comment'

Building the initial df.

In [62]:
params = {
    'subreddit': 'startrek',
    'size': 100
}

In [63]:
res = requests.get(url+subm_end, params)

In [64]:
res.status_code

200

In [65]:
data = res.json()

In [66]:
posts = data['data']

In [67]:
len(posts)

100

In [68]:
df = pd.DataFrame(posts)

In [69]:
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,url_overridden_by_dest,whitelist_status,wls,removed_by_category,media,media_embed,secure_media,secure_media_embed,distinguished,author_cakeday
0,[],False,DemiFiendRSA,None,[],None,text,t2_165g7s,False,False,...,https://deadline.com/2022/06/patrick-stewart-p...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,donthepunk,None,[],None,text,t2_70ko8pjj,False,False,...,NaN,all_ads,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Dangerous_Dac,None,[],None,text,t2_eq9t1mcg,False,False,...,NaN,all_ads,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,honeyfixit,None,[],None,text,t2_3vfmehh,False,False,...,NaN,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,casino_alcohol,None,[],None,text,t2_6icikfvu,False,False,...,NaN,all_ads,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df.shape

(100, 71)

In [71]:
# get last time 
df.iloc[-1]['created_utc']

1655785172

I referenced this [stackoverflow](https://stackoverflow.com/questions/28423964/for-loop-should-loop-9-times-instead-loops-once) to see using a counter to iterate through the for-loop 9 times (to get 1000 submissions in the df).

I needed [this website](https://techoverflow.net/2020/05/24/how-to-get-last-row-of-pandas-dataframe/) to remind me to use `.iloc[]` to get the last row (my previous stab at it having failed).

In [72]:
# I could start with an outer for loop that tests whether df is empty or not. if it is, it establishes the initial df
# else it runs the below

# Write a for-loop on a counter of 9, to have a total of 1000 posts.

for i in range(9): # I want to write my function so you can change the number of iterations in the function.
    
    params = {
        'subreddit': 'startrek', #ultimately, I want to make this a variable in the function.
        'size': 100,
        'before': df.iloc[-1]['created_utc']
    }
    res = requests.get(url+subm_end, params) #I want to make the +subm_end part a variable in the function,
                                                # default to subm_end, I think.
    #add a counter here, instead of printing the res.status_code, count the cycles and add to status code.
    if res.status_code == 200:
        print(res.status_code)
        data = res.json()
        posts = data['data']
        newdf = pd.DataFrame(posts)
        df = pd.concat([df, newdf], ignore_index = True) 

# Make next 100 posts a new df. Append them to the original df.
    else:
        print('ERROR: STATUS CODE NOT 200')

200
200
200
200
200
200
200
200
200


In [73]:
df.shape

(999, 73)

Why more columns? Why 999, not 1000? Maybe the first one gets overwritten somehow?

In [74]:
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,wls,removed_by_category,media,media_embed,secure_media,secure_media_embed,distinguished,author_cakeday,author_flair_background_color,author_flair_text_color
0,[],False,DemiFiendRSA,None,[],None,text,t2_165g7s,False,False,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,donthepunk,None,[],None,text,t2_70ko8pjj,False,False,...,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Dangerous_Dac,None,[],None,text,t2_eq9t1mcg,False,False,...,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,honeyfixit,None,[],None,text,t2_3vfmehh,False,False,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,casino_alcohol,None,[],None,text,t2_6icikfvu,False,False,...,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df = []

I got this error `SyntaxError: non-default argument follows default argument` on my initial attempt. I didn't realize the default arguments had to all come at the end. [This stackoverflow](https://stackoverflow.com/a/39942121) answer clarified that for me.

In [94]:
def build_reddit_df(subreddit, desired_size, endpoint = '/reddit/search/submission', limit = 100):
    '''
    NOTE: you must set your desired dataframe equal to this function to save the dataframe outside the function.
    
    This function is designed to use pushshift API to build a dataframe of specified size 
    filled with data from the specified subreddit. It starts with the most recent post and works backwards.

    df = name of dataframe you want to build

    endpoint = your desired endpoint. Defaults to '/reddit/search/submission' for submissions (main post) and
       '/reddit/search/comment' for comments at the time of the writing of this function (6/22/2022) 

    subreddit = the subreddit you'd like to scrape

    limit = the limit for number of posts that can be pulled at once. The default is 100, the maximum
        allowed at the time of the writing of this function (6/22/2022)

    desired_size = the total number of posts you'd like to have in the dataframe
    '''
    url = 'https://api.pushshift.io'+endpoint
    
    counter = 0
    
    fncdf = [] #establish with certainty that the new dataframe name is empty.
    
    for i in range(2):
   
        if len(fncdf) == 0:
            params = {
                'subreddit': subreddit,
                'size': limit
            }
            res = requests.get(url, params)
            if res.status_code == 200:
                data = res.json()
                posts = data['data']
                fncdf = pd.DataFrame(posts)
                counter += 1
            else:
                print(f'ERROR: status code not 200. Failure occured on loop number {counter+1}')

        else: # after the df has been established.
            for i in range(int(round(desired_size)/(limit))-1):
                params = {
                    'subreddit': subreddit,
                    'size': limit,
                    'before': fncdf.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)

                if res.status_code == 200:
                    data = res.json()
                    posts = data['data']
                    newdf = pd.DataFrame(posts)
                    fncdf = pd.concat([fncdf, newdf], ignore_index = True)
                    counter +=1

                else:
                    print('ERROR: status code not 200. Failure occured on loop number {counter+1}')
    
    return fncdf

In [120]:
startrek = build_reddit_df('startrek', 800)

In [96]:
startrek.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,preview,thumbnail_height,thumbnail_width,url_overridden_by_dest,media,media_embed,secure_media,secure_media_embed,distinguished,author_cakeday
0,[],False,Easy-Neighborhood-47,None,[],None,text,t2_5tavh4in,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,DemiFiendRSA,None,[],None,text,t2_165g7s,False,False,...,"{'enabled': False, 'images': [{'id': 'mG__QvaI...",93.0,140.0,https://deadline.com/2022/06/patrick-stewart-p...,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,donthepunk,None,[],None,text,t2_70ko8pjj,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Dangerous_Dac,None,[],None,text,t2_eq9t1mcg,False,False,...,"{'enabled': False, 'images': [{'id': '4mrCJMWT...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,honeyfixit,None,[],None,text,t2_3vfmehh,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
startrek.shape

(799, 71)

In [104]:
startrek[startrek['selftext'].value_counts()[:]>1]

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_72928/335038623.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  startrek[startrek['selftext'].value_counts()[:]>1]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).